In [2]:
%pylab inline
import pandas as pd
from sys import platform
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from geopy.geocoders import Nominatim
import folium
from datetime import datetime
from meteostat import Point, Daily

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [11]:
from datetime import datetime
from meteostat import Point, Hourly, units

# Set time period
start = datetime(2022, 8, 1, 1, 1, 1)
end = datetime(2022, 8, 1, 2, 1, 1)

# Create Point for DC
location = Point(38.72168084908155, -77.04777737365207)

# Get daily data
data = Hourly(location, start, end)
data = data.convert(units.imperial)
data = data.fetch()

In [10]:
data

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2000-01-01,45.0,33.1,55.9,0.000,0.000,NaN,4.2,NaN,1023.9,NaN
2000-01-02,55.9,44.1,68.0,0.000,0.000,182.0,12.7,NaN,1020.2,NaN
2000-01-03,60.1,52.0,68.0,0.000,0.000,177.0,10.9,NaN,1017.6,NaN
2000-01-04,60.1,48.0,71.1,1.220,0.000,211.0,15.9,NaN,1008.0,NaN
2000-01-05,42.1,35.1,48.0,0.000,0.000,328.0,14.1,NaN,1023.5,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-01-06,40.5,36.0,44.1,0.000,1.969,8.0,7.6,NaN,1012.7,NaN
2022-01-07,32.4,25.2,36.0,0.189,3.150,311.0,12.3,NaN,1018.7,NaN
2022-01-08,27.0,22.1,33.1,0.000,1.181,262.0,7.6,NaN,1035.6,NaN


In [5]:
import pandas as pd
import math

df = data.reset_index()
df['time'] = pd.to_datetime(df['time'])
df['month'] = df['time'].dt.month
df['year'] = df['time'].dt.year
df['decade'] = [math.floor(x/10) * 10 for x in df['year']]

In [6]:
temp = 90
df_summary = (
    df
    .query('tmax >= @temp')
    .groupby(['year'], as_index = False)
    .size()
    )

df_summary

,year,size
0,2000,12
1,2001,16
2,2002,56
3,2003,20
4,2004,11
5,2005,39
6,2006,36
7,2007,44
8,2008,35
9,2009,22


In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# rolling average
df_summary = (df_summary
                .assign(rolling = lambda df: df['size'].rolling(5, min_periods = 5).mean())
                .query('~rolling.isnull()')
                )

# linear regression
X = df_summary['year'].values.reshape(-1, 1)  
Y = df_summary['size'].values  
linear_regressor = LinearRegression()  

linear_regressor.fit(X, Y)  # perform linear regression

LinearRegression()

In [8]:
Y_pred = linear_regressor.predict(X)  # make predictions

df_summary['pred'] = Y_pred
df_summary

,year,size,rolling,pred
4,2004,11,23.0,31.935673
5,2005,39,28.4,33.211214
6,2006,36,32.4,34.486756
7,2007,44,30.0,35.762298
8,2008,35,33.0,37.037840
9,2009,22,35.2,38.313381
10,2010,67,40.8,39.588923
11,2011,50,43.6,40.864465
12,2012,53,45.4,42.140007
13,2013,35,45.4,43.415549
